In [ ]:
from fastai.vision.all import *
np.set_printoptions(suppress=True)

In [ ]:
proj_root = Path.cwd().parent
path = proj_root/'data'

In [ ]:
path.ls()

In [ ]:
dls = ImageDataLoaders.from_folder(path, train='train', valid='val', bs=32, item_tfms=Resize(256))

In [ ]:
dls.show_batch()

In [ ]:
learn = vision_learner(dls, resnet50, metrics=accuracy)
learn.fine_tune(5, cbs=EarlyStoppingCallback(monitor='accuracy', min_delta=0.1, patience=2))

In [ ]:
files = get_image_files(path/"labelbook")
test_file = files[-1]
test_file

In [ ]:

label, _, probs = learn.predict(test_file)
label, np.array(probs)

In [ ]:
dls.vocab

In [ ]:
np.array(probs).max()

In [ ]:
import hashlib
import json

In [ ]:
predictions_folder = proj_root/"predictions"
predictions_folder.mkdir(exist_ok=True)

In [ ]:
for file_path in files:
    with open(file_path, 'rb') as file_to_check:
        # read contents of the file
        data = file_to_check.read()    
        # pipe contents of the file through
        md5 = hashlib.md5(data).hexdigest()

    pred_label, _, probs = learn.predict(file_path)
    confidence = np.array(probs).max()
    #true_label = class_names[ test_labels[index] ]
    file_name = file_path.stem
    
    json_string = f'{{ "annotation": {{ "inference": {{ "label": "{pred_label}", "confidence": {confidence} }} }}, "data-object-info": {{ "md5": "{md5}" }} }}'
    # print(json_string)
    json_data = json.loads(json_string)
    with open(predictions_folder/f'{file_name}.json', 'w', encoding='utf-8') as f:
        json.dump(json_data, f, ensure_ascii=False, indent=4)
        